경로지정과 csv 읽는 파일만 변경해주면 돼

누적 저장 가능

200개씩 알아서 끊어서 크롤링해줌

크롤링 성공한 가게 수

크롤링 실패한 가게 수

총 크롤링한 리뷰 개수 마지막에 포함시킴


변경해야할 부분을 컨트롤 f 로 검색하면 뭘 변경해야할지 알 수 잇음

모르겠으면 물어보셔유

In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import time
import datetime
import os
import re

crawl = 400 ## 변경해야할 부분

# 브라우저 옵션 설정
options = Options()
options.add_argument("window-size=1920x1080")
service = ChromeService(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# 데이터 불러오기
df_filtered = (
    pd.read_csv('서울시 휴게음식점 인허가 정보-2.csv')
    .query('업태구분명 in ["다방", "커피숍", "떡카페"] and 상세영업상태명 == "영업"')
    [['사업장명', '도로명주소']]
    .fillna({'도로명주소': ''})
)

df_filtered['도로명주소'] = (
    df_filtered['도로명주소']
    .apply(lambda x: ' '.join(x.split(',')[0].split()[:4]))
)

records = df_filtered.to_dict(orient='records')
now = datetime.datetime.now()

# 누적 저장을 위한 CSV 경로 및 컬럼 정의
csv_path = os.path.expanduser('~/Desktop/naver_review.csv')
if os.path.exists(csv_path):
    df = pd.read_csv(csv_path, encoding='utf-8-sig')

    if not df.empty:
        last_row = df.iloc[-1]  # 마지막 행
        total_review = last_row['총 리뷰 개수']
        already_crawled =  last_row['최종 크롤링 개수']
        print("최종 크롤링 개수:", already_crawled)
    else:
        print("CSV 파일은 있지만 데이터가 없습니다.")
else:
    already_crawled = 0 ## 변경해야할 부분
    total_review = 1
    print("CSV 파일이 존재하지 않습니다.")

columns = ['공공데이터 가게명',
           '크롤링 가게명',
           '카테고리',
           '시간',
           '평점',
           '출처',
           'content',
           'date',
           'revisit',
           '방문목적',
           '동반자',
           '태그',
           '선택 수',
           '편의시설',
           '최종 크롤링 개수',
           '총 리뷰 개수']
review_data = []
success = 0
fail = 0
review_cnt = 0
crawl_end = already_crawled+crawl
if crawl_end > 5324: #변경해야할 부분
    crawl_end = 5324

# 크롤링 시작
for row in records[already_crawled:crawl_end]:
    restaurant = row['사업장명']
    address = row['도로명주소']
    driver.get("https://m.map.naver.com/#search")
    time.sleep(3)

    try:
        search_input = driver.find_element(By.XPATH, '//input[contains(@placeholder, "검색")]')
        search_input.click()
        search_input.send_keys(restaurant)
        search_input.send_keys(Keys.ENTER)
    except Exception as e:
        print(f"❗ 검색창 오류: {e}")
        continue

    time.sleep(3)
    found_matching_address = False

    try:
        search_restaurant_list = driver.find_elements(By.XPATH, "//div[contains(@class, '_item_info_wrap_')]")
        for a in search_restaurant_list:
            try:
                add_el = a.find_element(By.XPATH, ".//button[contains(@class,'_item_address_')]")
                address_extracted = add_el.text.strip()
            except:
                continue

            if address not in address_extracted:
                continue

            found_matching_address = True

            try:
                a_href_el = a.find_element(By.XPATH, ".//a[contains(@href,'/place/') and contains(@href,'/home')]")
                a_href = a_href_el.get_attribute("href")
            except:
                a_href = ""

            try:
                name_el = a.find_element(By.XPATH, ".//strong[contains(@class,'_item_name_')]")
                store_name = name_el.text.strip()
            except:
                store_name = ""

            try:
                cat_el = a.find_element(By.XPATH, ".//em[contains(@class,'_item_category_')]")
                category = cat_el.text.strip()
            except:
                category = ""

            a_href_el.click()
            break

        if not found_matching_address:
            fail=fail+1
            # print(f"❗ {restaurant} - 주소 매칭 실패, 건너뜀")
            continue
        success = success+1
        time.sleep(3)
        # print(store_name)

        try:
            # XPath로 조건에 맞는 a 태그 선택
            button = driver.find_element(By.XPATH, '//a[contains(@class, "gKP9i RMgN0") and @aria-expanded="false"]')
            button.click()
            time.sleep(2)
        except Exception as e:
            print(f"❗ 버튼 클릭 실패: {e}")
            pass

        try:
            # rows = driver.find_elements(By.XPATH, '//div[contains(@class, "w9QyJ")]')
            rows = driver.find_elements(By.XPATH, '//*[@class="w9QyJ"]')
            hours_list = []

            for row in rows:
                # 요일 추출 (없으면 "매일")
                try:
                    day = row.find_element(By.XPATH, './/span[contains(@class, "i8cJw")]').text.strip()
                except:
                    day = "매일"

                # 시간 추출 (없으면 "정보 없음")
                time_range_elements = row.find_elements(By.XPATH, './/div[contains(@class, "H3ua4")]')
                time_range = time_range_elements[0].text.strip() if time_range_elements else "정보 없음"

                hours_list.append(f"{day} {time_range}")

            time_combined = " | ".join(hours_list)
            # print("🕒 영업시간:", time_combined)

        except Exception as e:
            print(f"❗ 영업 시간 전체 추출 실패: {e}")
            time_combined = ""
            pass
        # 정보탭 클릭
        try:
            information_tab = driver.find_element(By.XPATH, "//a[contains(@href,'/information')]")
            information_tab.click()
        except Exception as e:
            print(f"❗ 정보 탭 오류: {e}")
            pass
        time.sleep(3)
        # place_section_content 하위의 owG4q 요소들 선택
        try:
            elements = driver.find_elements(By.XPATH, '//ul[contains(@class, "JU0iX")]//div[@class="owG4q"]')
            facility_data = [el.text.strip() for el in elements if el.text.strip()]
        except:
            facility_data = []
            pass
        # print(facility_data)
        time.sleep(3)

        try:
            review_tab = driver.find_element(By.XPATH, "//a[contains(@href,'/review')]")
            driver.execute_script("arguments[0].scrollIntoView(true);", review_tab)
            time.sleep(1)
            driver.execute_script("arguments[0].click();", review_tab)
            time.sleep(2)
            # print("✅ 리뷰 탭 클릭 성공")
        except Exception as e:
            print(f"❗ 리뷰 탭 클릭 실패: {e}")
            continue

        time.sleep(3)

        try:
            rating = driver.find_element(By.XPATH, "//span[contains(@class,'PXMot') or contains(@class,'_1Y6hi')]").text.strip()
        except:
            rating = ""

        tag_data = []
        # 태그 더보기 반복 클릭
        while True:
            try:
                more_button = driver.find_element(By.XPATH, '//span[contains(@class, "YqDZw")]')
                driver.execute_script("arguments[0].click();", more_button)
                time.sleep(1)
            except:
                break

        # 태그 요소 수집
        tag_elements = driver.find_elements(By.XPATH, '//span[contains(@class, "t3JSf")]')
        for tag in tag_elements:
            try:
                tag_name = tag.text.strip()
                tag_count_element = tag.find_element(By.XPATH, './following-sibling::span')
                tag_count = int(re.sub(r'[^0-9]', '', tag_count_element.text.strip()))
                tag_data.append((tag_name, tag_count))
            except:
                continue

        tag_data = sorted(tag_data, key=lambda x: x[1], reverse=True)  # 전체 태그 수집
        # 리뷰 수집
        all_reviews = []
        review_limits = {"추천": 5, "최신": 5}

        for source_label in ["추천", "최신"]:
            if source_label == "최신":
                try:
                    latest_button = driver.find_element(By.XPATH, "//a[@role='option' and @aria-selected='false']")
                    latest_button.click()
                    time.sleep(5)
                except:
                    print(f"❗ 최신순 버튼 클릭 오류")
                    continue

            reviews = driver.find_elements(By.XPATH, '//li[contains(@class,"place_apply_pui")]')
            count = 0

            for r in reviews[:review_limits[source_label]]:
                try:
                    content = r.find_element(By.XPATH, './/div[contains(@class,"pui__vn15t2")]').text.strip()
                    date = r.find_element(By.XPATH, './/span[contains(@class,"pui__gfuUIT")]/time').text.strip()

                    revisit_elements = r.find_elements(By.XPATH, './/span[contains(@class,"pui__gfuUIT")]')
                    revisit = revisit_elements[1].text.strip() if len(revisit_elements) > 1 else ''
                    if revisit:
                        revisit = int(revisit.replace(",", "")[:-5])

                    try:
                        tag_expand_button = r.find_element(By.XPATH, './/a[contains(@class,"pui__jhpEyP")]')
                        driver.execute_script("arguments[0].click();", tag_expand_button)
                        time.sleep(3)
                    except:
                        pass

                    try:
                        div = r.find_element(By.XPATH, './/div[contains(@class, "pui__-0Ter1")]')
                        spans = div.find_elements(By.TAG_NAME, 'span')
                        span_texts = [span.text.strip() for span in spans]
                        visit_tag = span_texts[3].strip() if len(span_texts) >= 3 else "태그 없음"
                        mate_tag = span_texts[4].strip() if len(span_texts) >= 4 else "태그 없음"
                    except:
                        visit_tag = "태그 없음"
                        mate_tag = "태그 없음"

                    all_reviews.append([
                        restaurant, store_name, category, time_combined, rating, source_label,
                        content, date, revisit,  visit_tag, mate_tag, already_crawled+crawl,total_review+review_cnt
                    ])
                    count += 1
                    review_cnt = review_cnt+1
                except:
                    continue

        max_len = max(len(all_reviews), len(tag_data), len(facility_data))
        # ✅ 태그가 10개 미만일 경우 빈 값으로 채우기
        tag_data += [("", "")] * (max_len - len(tag_data))
        facility_data += [""] * (max_len - len(facility_data))
        all_reviews += [
            [restaurant, store_name, category, time_combined, rating, '', '', '', '', '', '', already_crawled + crawl, total_review + review_cnt]
        ] * (max_len - len(all_reviews))

        for i in range(max_len):
            review = all_reviews[i]
            tag_name, tag_count = tag_data[i]
            facility = facility_data[i]

            review_data.append([
                *review[:11],      # 기본 리뷰 정보 (0~10)
                tag_name, tag_count,
                facility,          # 편의시설
                review[11], review[12]  # 크롤링 수, 리뷰 수
            ])
    except Exception as e:
        print(f"❗ 가게 검색 중 오류: {e}")

driver.quit()

# 누적 저장 처리
df_new = pd.DataFrame(review_data, columns=columns)

if os.path.exists(csv_path):
    df_old = pd.read_csv(csv_path, encoding='utf-8-sig')
    df_combined = pd.concat([df_old, df_new], ignore_index=True)
else:
    df_combined = df_new

# 저장  ## 변경해야할 부분
df_combined.to_csv(csv_path, index=False, encoding='utf-8-sig')
print(f"크롤링 성공 개수 : {success}, 크롤링 실패 개수 : {fail}, 총 수집한 리뷰 개수 : {review_cnt}")
print(f"📁 리뷰 데이터를 누적 저장했습니다: {csv_path}")